# Install and import libraries

In [ ]:
%pip install pydot
%pip install tensorflow
%pip install scikit-learn

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,
    METADATA_PATH,
)

from constants import CLASSES
num_classes = len(CLASSES)

--- NeuroSyn Physio Constants Loaded (v3 - Compatibility Names) ---
Myo Address: DD:31:D8:40:BC:22
Data Input Path: model-v4\data
Model Path: model\physio_model.h5
Metadata Path: model\physio_metadata.pkl
Collection Time per Rep (s): 5
Repetitions per Exercise: 5
Number of Classes: 9
Classes Map: {0: 'Rest', 1: 'Wrist Flexion', 2: 'Wrist Extension', 3: 'Elbow Flexion', 4: 'Elbow Extension', 5: 'Hand Close', 6: 'Hand Open', 7: 'Forearm Pronation', 8: 'Forearm Supination'}
-----------------------------------


# Read the files in the data dir

In [2]:
# Read all of the files in the data folder
files_in_folder = Path(DATA_INPUT_PATH).glob("*.csv")

files = [x for x in files_in_folder]
print([file for file in files])

[WindowsPath('data/physio_emg_imu_data_20250429_001144.csv'), WindowsPath('data/physio_emg_imu_data_20250429_174128.csv'), WindowsPath('data/physio_emg_imu_data_20250429_175521.csv'), WindowsPath('data/physio_emg_imu_data_20250430_002614.csv')]


## Convert .csv(s) to dataframes and concatenate

In [3]:
# Read the data from the files
dfs = []
print("Looking in:", DATA_INPUT_PATH)
print("Found CSVs:", list(Path(DATA_INPUT_PATH).glob("*.csv")))

for file in files:
    df = pd.read_csv(str(file))
    dfs.append(df)
        
# Convert the data to a DataFrame
df = pd.concat([x for x in dfs], axis=0)

# after concatenating dfs…
columns = ["gesture_id"] \
        + [f"s{i}" for i in range(1,9)] \
        + ["quat_w","quat_x","quat_y","quat_z"]
df = df[columns]

print(df.head())

# Before removing duplicates
print(f"Shape of dataframe before removing duplicates {df.shape}")
# Remove duplicates
df = df.drop_duplicates()
print(f"Shape of dataframe after removing duplicates {df.shape}")

Looking in: data
Found CSVs: [WindowsPath('data/physio_emg_imu_data_20250429_001144.csv'), WindowsPath('data/physio_emg_imu_data_20250429_174128.csv'), WindowsPath('data/physio_emg_imu_data_20250429_175521.csv'), WindowsPath('data/physio_emg_imu_data_20250430_002614.csv')]
   gesture_id   s1   s2  s3  s4  s5  s6  s7   s8    quat_w    quat_x  \
0           0  320  214  81  41  38  26  26   69  0.875610  0.454102   
1           0  347  217  81  36  36  28  31  115  0.875610  0.454102   
2           0  224  131  55  37  35  27  28  116  0.875671  0.454041   
3           0  207  114  51  35  31  27  31  122  0.875671  0.453918   
4           0  182  100  45  37  30  27  30  118  0.875671  0.453979   

     quat_y    quat_z  
0 -0.153137 -0.059937  
1 -0.153137 -0.059937  
2 -0.153076 -0.059937  
3 -0.153198 -0.059692  
4 -0.153381 -0.059326  
Shape of dataframe before removing duplicates (42037, 13)
Shape of dataframe after removing duplicates (42037, 13)


In [4]:
# --- 3) Sliding‐window RMS feature extraction ---
import numpy as np
from constants import WINDOW_SIZE, WINDOW_STEP

# build channel list in correct order
channels = [f"s{i}" for i in range(1,9)] + ["quat_w","quat_x","quat_y","quat_z"]
raw = df[channels].values          # shape = [total_samples, 12]
labels = df["gesture_id"].values   # shape = [total_samples,]

X, y = [], []
for start in range(0, len(raw) - WINDOW_SIZE + 1, WINDOW_STEP):
    window = raw[start : start + WINDOW_SIZE]     # shape = [WINDOW_SIZE, 12]
    rms_feat = np.sqrt(np.mean(window**2, axis=0))  # compute RMS per channel
    X.append(rms_feat)
    # assign the window’s “center” label
    center_idx = start + WINDOW_SIZE//2
    y.append(labels[center_idx])

X = np.array(X)   # shape = [n_windows, 12]
y = np.array(y)   # shape = [n_windows,]

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (839, 12)
y shape: (839,)


## Scale, split, and one-shot encode RMS feature windows

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# ---- scale features ----
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---- one-hot labels ----
y_cat = to_categorical(y, num_classes)

# ---- train / test split ----
# stratify=y ensures each class is proportionally represented
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train:", X_train.shape, "  y_train:", y_train.shape)
print("X_test: ", X_test.shape, "  y_test: ",  y_test.shape)


X_train: (671, 12)   y_train: (671, 9)
X_test:  (168, 12)   y_test:  (168, 9)


## Build, compile & train a simple dense classifier

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping]
)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.3%}")

Epoch 1/100


C:\Users\Kavish Krishnakumar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.1202 - loss: 2.9548 - val_accuracy: 0.2444 - val_loss: 2.1199
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1874 - loss: 2.3590 - val_accuracy: 0.3037 - val_loss: 2.0547
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2822 - loss: 1.9224 - val_accuracy: 0.4741 - val_loss: 1.9932
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3949 - loss: 1.7796 - val_accuracy: 0.5037 - val_loss: 1.9376
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4944 - loss: 1.5185 - val_accuracy: 0.5037 - val_loss: 1.8797
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5031 - loss: 1.4912 - val_accuracy: 0.5407 - val_loss: 1.8181
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5563 - loss: 1.3083 - val_accuracy: 0.6000 - val_loss: 1.7488
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6170 - loss: 1.1975 - val_accuracy: 0.6370 - val_loss: 1.6824
Epoch 9/100

## Save the model and the scalar + feature list

In [7]:
# make sure the folder exists
(Path(MODEL_PATH).parent).mkdir(parents=True, exist_ok=True)

model.save(MODEL_PATH)

# pickle the scaler and the channel names
with open(METADATA_PATH, "wb") as f:
    # channels list we used for X is:
    feature_names = [f"s{i}" for i in range(1,9)] + ["quat_w","quat_x","quat_y","quat_z"]
    pickle.dump((scaler, feature_names), f)

print("Saved model to", MODEL_PATH)
print("Saved metadata to", METADATA_PATH)

Saved model to model\physio_model.h5
Saved metadata to model\physio_metadata.pkl


In [8]:
import os
from constants import DATA_INPUT_PATH

print("Files in data folder:", os.listdir(DATA_INPUT_PATH))


Files in data folder: ['artemis_archive', 'convert_old_data.py', 'physio_emg_imu_data_20250429_001144.csv', 'physio_emg_imu_data_20250429_174128.csv', 'physio_emg_imu_data_20250429_175521.csv', 'physio_emg_imu_data_20250430_002614.csv']


In [9]:
# ─── Demo on latest collected Physio CSV ──────────────────────────────────────
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from pathlib import Path

from constants import DATA_INPUT_PATH, MODEL_PATH, METADATA_PATH, CLASSES

# 1) locate the CSV folder and pick the newest file
data_dir = Path(DATA_INPUT_PATH)
csv_files = list(data_dir.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"No CSVs found in {data_dir.resolve()}")

# pick by modification time
csv_path = max(csv_files, key=lambda p: p.stat().st_mtime)
print("Using CSV:", csv_path.name)

# 2) load it
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} rows from {csv_path.name}")

# 3) pick one sample per class
demo_df = (
    df
    .groupby("gesture_id", group_keys=False)
    .apply(lambda g: g.sample(1, random_state=0))
    .reset_index(drop=True)
)
demo_df["true_name"] = demo_df["gesture_id"].map(CLASSES)
print("\nDemo samples:")
display(demo_df[["gesture_id","true_name"]])

# 4) load scaler + feature list
with open(METADATA_PATH, "rb") as f:
    scaler, feature_names = pickle.load(f)
print("\nFeatures:", feature_names)

# 5) load model
model = load_model(MODEL_PATH)

# 6) prepare & predict
X_demo = demo_df[feature_names].values
X_scaled = scaler.transform(X_demo)
preds = model.predict(X_scaled, verbose=0)
pred_ids = np.argmax(preds, axis=1)
demo_df["pred_id"]   = pred_ids
demo_df["pred_name"] = demo_df["pred_id"].map(CLASSES)

# 7) show results
results = demo_df[[
    "gesture_id","true_name","pred_id","pred_name"
]].rename(columns={"gesture_id":"true_id"})
print("\nPredictions:")
display(results)

acc = (demo_df["pred_id"] == demo_df["gesture_id"]).mean()
print(f"\nDemo accuracy (1-sample/class): {acc:.0%}")


Using CSV: physio_emg_imu_data_20250430_002614.csv
Loaded 10533 rows from physio_emg_imu_data_20250430_002614.csv

Demo samples:


C:\Users\Kavish Krishnakumar\AppData\Local\Temp\ipykernel_31260\3925246817.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(1, random_state=0))


,gesture_id,true_name
0,0,Rest
1,1,Wrist Flexion
2,2,Wrist Extension
3,3,Elbow Flexion
4,4,Elbow Extension
5,5,Hand Close
6,6,Hand Open
7,7,Forearm Pronation
8,8,Forearm Supination



Features: ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 'quat_w', 'quat_x', 'quat_y', 'quat_z']

Predictions:


,true_id,true_name,pred_id,pred_name
0,0,Rest,5,Hand Close
1,1,Wrist Flexion,1,Wrist Flexion
2,2,Wrist Extension,2,Wrist Extension
3,3,Elbow Flexion,7,Forearm Pronation
4,4,Elbow Extension,4,Elbow Extension
5,5,Hand Close,5,Hand Close
6,6,Hand Open,0,Rest
7,7,Forearm Pronation,0,Rest
8,8,Forearm Supination,8,Forearm Supination



Demo accuracy (1-sample/class): 56%
